Устанавливаем и импортируем библиотеки

In [5]:
!pip install -q -U kaggle_environments

In [1]:
from kaggle_environments import make, evaluate

Вспомогательная функция для определения победителя в одном раунде

In [18]:
%%writefile utils.py
import math
def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

Overwriting utils.py


## Создание агентов
Создаем простых агентов показывающих только камень(0), только бумагу(1) или только ножницы (2)

In [20]:
%%writefile rock_agent.py

def rock_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


In [21]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 0

Overwriting paper_agent.py


In [22]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


Агент возвращающий случайный символ

In [23]:
%%writefile rand_agent.py

import random

def rand_agent(observation, configuration):
    return random.randrange(0, configuration.signs)

Overwriting rand_agent.py


Агент копирующий последний ход соперника

In [7]:
%%writefile copy_opponent_agent.py

import random

def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent_agent.py


Этот агент смотрит если в предыдущей игре он проиграл, то меняет стратегию так, что бы она была выигрышной для предыдущего хода соперника. В противном случае ничего не меняет

In [8]:
%%writefile last_react_agent.py

import random
from utils import get_score

last_react_action = None

def reactionary(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs
    return last_react_action



Overwriting last_react_agent.py


Этот агент меняет свой выбор на следующий по порядку (камень-бумага-ножницы), в случае если в прошлый раз проиграл.

In [9]:
%%writefile reward_agent.py

my_last_action = None
last_reward = 0

def reward(observation, configuration):
    global my_last_action, last_reward
    if observation.step == 0:
        my_last_action = 0
    elif observation.reward <= last_reward:
         my_last_action = ( my_last_action + 1) % configuration.signs
    last_reward = observation.reward
    return  my_last_action

    

Overwriting reward_agent.py


Агент который в зависимости от исхода предыдущего хода меняет свой выбор. В случае победы через один от своего последнего ( в ряду камень-бумага-ножницы), в остальных случаях на противоположный последнему выбору противника.

In [10]:
%%writefile last_count_react_agent.py
import random
from utils import get_score

last_counter_action = None

def counter_reactionary(observation, configuration):
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs
    return last_counter_action



Overwriting last_count_react_agent.py


Агент который каждый новый ход меняет свой выбор на следующий

In [11]:
%%writefile step_agent.py
def step(observation, configuration):
    return observation.step % configuration.signs

Overwriting step_agent.py


Агент который меняет свой выбор на следующий раз в три хода

In [12]:
%%writefile step3_agent.py

flag = 0
my_action = 0

def step(observation, configuration):
    global flag, my_action
    if flag == 4:
        my_action = ( my_action + 1) % configuration.signs
        flag = 0
    flag += 1
    return my_action

Overwriting step3_agent.py


Агент который меняет свой выбор через один (в ряду камень-бумага-ножницы) каждый ход.

In [13]:
%%writefile step_2_agent.py

my_action = 0

def step(observation, configuration):
    global my_action
    my_action = ( my_action + 2) % configuration.signs
    return my_action

Overwriting step_2_agent.py


Этот агент действует исходя из истории поведения соперника. Делает свой выбор так что бы выиграть у самого частого выбора соперника.

In [14]:
%%writefile hist_agent.py
import random


action_histogram = {}


def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Overwriting hist_agent.py


## Организуем турнир
Составим список стратегий

In [15]:
agents = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'step_agent.py', 'step3_agent.py'
       , 'step_2_agent.py', 'hist_agent.py', 'last_react_agent.py', 'last_count_react_agent.py', 
        'reward_agent.py', 'copy_opponent_agent.py', 'rand_agent.py'
         ]

Введем функцию раздающую очки за победу 1 - победа, 0 - ничья, -1 - поражение

In [16]:
def sign(x):
    if x > 0:
        return 1
    elif x == 0:
        return 0
    else:
        return -1

Сам турнир

In [17]:
res_table = [0 for i in range(12)] 
for id1, first in enumerate(agents):
    for id2, second in enumerate(agents): # Проходим по всем стратегиям
        #Организуем "встречи"
        res = evaluate(
            "rps", 
            [first, second], 
            configuration={"episodeSteps": 100}  
            )
       #Записываем результаты
        res_table[id1-1] += sign(res[0][0])
        res_table[id2-1] += sign(res[0][1])
    

Определяем победителя


In [108]:
max_result = max(res_table)
victory_id = res_table.index(a)
print('Победила стратегия ' + agents[victory_id][:-9])

Победила стратегия last_count_react
